# Лабораторная работа 2
### Выполнил: Самсонов Олег Игоревич

Начать хотелось бы с неформальной шутки: я, как человек играющий в лигу легенд до глубины души оскорблен выбором доты 2 в качестве предметной области лабораторной работы и кегла, и теперь жду, что третья работа должна быть посвящена Lol.

### Мини введение
Так получилось, что в этой лабе я сделал упор на то чтобы попробовать разные модели. По итогу я попробовал SVM, нейросети, логистическую и линейную регрессии и немного деревьев с бустингами. Мне хотелось увидеть как поведут себя различные модели на задаче, но из-за этого я "погнался за двумя зайцами, и не одного не поймал". Не одна модель так и не была доведена до совершенства, а анализ и обработка данных была отложена даже не на 2й, а на 3й план. Вряд ли я смогу прямо показать все свои эксперименты тут, но постараюсь продемонстрировать основное и самое как на мой взгляд важное.

### Сюда я вставил некоторые функции которые будут использоваться позже

In [12]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import numpy as np
random_seed = 42

def test_by_nn(nn,loader, is_many_vals = False):
    test_preds = np.array([])

    if is_many_vals:
        for x_batch, y in loader:
            predictions = nn(x_batch)
            test_preds = np.append(test_preds, predictions.argmax(dim=-1).numpy())
    else:
        for x_batch, y in loader:
            predictions = nn(x_batch)
            print(predictions)
            predictions = torch.transpose(predictions, 0, 1)[0]
            predictions = predictions.detach().numpy()
            test_preds = np.append(test_preds, predictions)
    return test_preds

def delete_not_x_y(_data : pd.DataFrame):
    data = _data.copy()
    for cat in data.columns:
        if cat != "target":
            if not "_x" in cat and not "_y" in cat or "xp" in cat:
                data = data.drop(cat, axis = 1)
    return data

def delete_outliers(_data):
    data = _data.copy()
    data = data.where(data["game_time"] < 10000000)
    return data

def preprocess_trainsvmxy(_data, is_remove_outliers, is_split = True):
    data = _data.copy()
    data = data.drop("match_id", axis = 1)
    if is_remove_outliers:
        data = delete_outliers(data)
    data = inner_preprocesssvmxy(data)
    if is_split:
        return split_train(data)
    else:
        return data


def inner_preprocess(_data : pd.DataFrame):
    data = _data.copy()
    data = data.dropna()
    return data

class MyTensorDataset(torch.utils.data.Dataset):
    def __init__(self, *args, **kwargs):
        assert len(args) > 0
        len_base = len(args[0])
        
        for i in args:
            assert len(i) == len_base
        
        self.data = args
        
    def __len__(self, ):
        return len(self.data[0])
        
    def __getitem__(self, idx):
        return [t[idx] for t in self.data]

def inner_preprocesssvmxy(_data):
    data = _data.copy()
    data = delete_not_x_y(data)
    print('x')
    data = data.dropna()
    return data

def preprocess_train(_data, is_remove_outliers, is_split = True, make_opposite_target = False):
    data = _data.copy()
    data = data.drop("match_id", axis = 1)
    if is_remove_outliers:
        data = delete_outliers(data)
    data = inner_preprocess(data)
    if is_split:
        return split_train(data, make_opposite_target)
    else:
        return data
    
def split_train(_data, make_opposite_for_targets = False):
    raw_train, raw_test = train_test_split(_data, test_size=0.2, random_state=random_seed)
    Y_train = pd.DataFrame(raw_train['target'])
    X_train = raw_train.drop('target', axis = 1)
    Y_test = pd.DataFrame(raw_test['target'])
    X_test = raw_test.drop('target', axis = 1)
    Y_train["target"] = Y_train['target'].astype(int)
    Y_test["target"] = Y_test['target'].astype(int)
    if make_opposite_for_targets:
        Y_train["targetOpposite"] = Y_train["target"].apply(lambda x : 1 if x == 0 else 0)
    return X_train, Y_train, X_test, Y_test

### EDA (исследовательский анализ данных)

Из-за количества данных анализ в форме таблиц или графиков сделать очень трудно. Да и предметная область достаточно сложная. Поэтому большую часть данных лично я пытался интерпретировать либо из знаний игрового жанра, либо из общей интуиции. В доту я наиграл всего часов 50, поэтому не уверен что все мои человеческие выводы хоть чуть-чуть близки к реальности.
Всего в датасете 243 фичи. Насколько я понял, они представляют собой объект некоторой игровой сессии, а точнее временного среза этой сессии в случайный момент времени.
Из них в том числе, так скажем, общие фичи:
1)match_id - id матча. Очевидно не значим
2)game_time - время которое длится игра, по моим ощущениям достаточно значимый параметр, ведь в начале игре влияние всех остальных параметров сильно слабеет на исход игры
3)game_mode - странный, категориальный параметр. Есть ощущение что это тип игры в том смысле что "нормал", "ранкед", "турбо". Честно сказать доказательств этому у меня нет, во всяком случае я считал его достаточно значимым.
4)lobby_type - честно говоря, не знаю. Вот прям совсем.
objectives_len -
5)chat_len - видимо, количество сообщений в чате. В целом это имеет определенное влияние на игру, но вряд ли его можно как-то легко интерпретировать.

После идут параметры героев. Всего героев 10, и видимо их порядок не соответствует ролям в игре(хотя во всяком случае я эти ваше "тройки", "коры" в доте не понимаю).
Для каждого героя есть параметры:
1)hero_id - id героя, скорее категориальная фича. Честно сказать, я считаю что в датасете маловато данных для выявления каких либо зависимостей синергий или контрпиков между героями. Но во всяком случае я обычно оставлял этот столбец, иногда разделяя его one hot encoderom(точнее getdummies, но не важно)
2)kills - киллы героя. Я интерпретировал его как важный
3)deaths - тоже самое что и киллы, но наоборот
4)assists - еще одно к тому же
5)denies - я бы сказал, что мало того что денай крипов имеет смысл "даю меньше денег врагам", так еще и может говорить о скилле игрока: слабый игрок много не заденаит
6)gold - деньги - основной способ получить примущество над противником, значимо
7)lh - без понятия =)
8)xp - опыть героя, тоже значим
9)health - в комбинации с временем игры может быть значимо, мало хп в конце игры вероятно = поражению
10)max_health - думаю не очень значимо, но я оставлял
11)max_mana - думаю не очень значимо, но я оставлял
12)level - значимо
13)x - о х и y будет история чуть позже в экспериментах
14)y - о х и y будет история чуть позже в экспериментах
15)stuns - слабо, но думаю значимо
16)creeps_stacked - слабо, но думаю значимо
17)camps_stacked - слабо, но думаю значимо
18)rune_pickups - слабо, но думаю значимо
19)firstblood_claimed - слабо, но думаю значимо
20)teamfight_participation - слабо, но думаю значимо
21)towers_killed - на самом деле, в долгих играх думаю одно из самых важных фичей. Все таки игра выигрывает не киллами, а сносом строений
22)roshans_killed - значимо
23)obs_placed - слабо, но думаю значимо(может быть такая же ситуация как и с денаями)
24)sen_placed - слабо, но думаю значимо(может быть такая же ситуация как и с денаями) 

### Preprocessing (подготовка данных)

В этот раз я в только удалял выбросы, и то только один вид - в датаесете есть игры невероятно аномальной длины по времени. Их я всего и выбрасывал.

### Model & training (Выбор модели и её обучение)

В этом разделе описываете модель и ставите эксперименты по обучению.

Если вы ставили много экспериментов, приведите их в хронологическом порядке чтобы мы увидели эволюцию ваших рассуждений.

#### Эксперимент 1 SVM

Начал я свои эксперименты с SVM. Задача классификации как никак. Пользовался я в основном ядром rbf. 

In [ ]:
# Просто из примера одна из первых моих SVM моделей
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
train = train.where(train["game_time"] < 10000000)
train = train.drop("match_id", axis = 1)
train = train.dropna()
from sklearn.model_selection import train_test_split
random_seed = 42
raw_train, raw_test = train_test_split(train, test_size=0.33, random_state=random_seed)
Y_train = pd.DataFrame(raw_train['target'])
X_train = raw_train.drop('target', axis = 1)
Y_test = pd.DataFrame(raw_test['target'])
X_test = raw_test.drop('target', axis = 1)
Y_train["target"] = Y_train['target'].astype(int)
Y_test["target"] = Y_test['target'].astype(int)
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, Y_train)
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict(X_test))}")

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Score : 0.5168732125834128
ROC-AUC score : 0.5


In [ ]:
model.predict(X_test)

array([1, 1, 1, ..., 1, 1, 1])

Как мы можем видеть, свм без параметров и особого препроцессинга выдает только 1. Что вообще имеет смысл - шанс угадать как раз где-то 50 процентов. Но нас это конечно не устраивает. Но. Хорошего результат я от SVM я так и не добился. Я пробовал частично выкидывать данные, менять их на категориальные(хотя тогда я еще об этом особо не думал), и даже как-то чисто из интереса запустил гридсерч который работал 6 часов.
Конечно у меня были модели, которые выдавали скор в 0.6, но бейс лайн никто так и не пробил(это в основном были модели с очееень маленькой С, и с гаммой равной scale, т.е. 1 / (n_features * X.var()), где var - дисперсия элементов)
У меня был всего интересный эксперимент, который я хотел бы описать.
Хотя это и не имело особого смысла, я как-то попробовал построить корреляционную матрицу(не всех всех параметров, а кусками. Всего вышло 11 таблиц, одна общая и по одной на каждого героя). И что было забавно, самая большая корреляция с таргетом была у X и Y. И хотя я конечно понимал, что в данном случае это не о чем не говорит, я попробовал загнать в SVM только x и y.
Иииии на самом деле это же имеет определенный смысл. Если задуматься, то отобрав все моменты игр, в который они по времени были близки к концу - скорее всего координаты героев побеждающей команды будут близки к координатам нексуса проигравшей. Конечно, вероятно таких игр в датасете совсем нет, и в целом в постановке решаемой задачи это не имеет смысла. Я решил попробовать. Давайте посмотрим что получится:

In [ ]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = preprocess_trainsvmxy(train, True)
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
model = SVC(kernel='rbf', C=1)
model.fit(X_train, Y_train)
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict(X_test))}")

x


c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Score : 0.6557164068299925
ROC-AUC score : 0.6510472241763213


Это признаться даже не лучший результат SVM, мне прото забавно с того что по х и y можно выдать предсказательную мощь больше 0.5. Так что пусть будет.

#### Эксперимент 2 Нейросети

Вот это было интересно. Я пару дней занимался буквально только тем что учил нейросети. И буду честен - видимо я очееееень плох в этом. Всего у меня было около 5 типов нейросетей: те что выдают только 1, те что выдают только 0, те что выдают только значения близкие к 0.5, те что выдают только 1 или 0.5, и адекватные - но даже у них скор был не очень высок - около 0.5.
Я пробовал всякое - сначала, я пытался сделать простую сеть, из нескольких линейных слоев и сигмоиды в качестве финальной функции активации.
Тогда я использовал BCELOSS(бинарный кросс энтропия) или Hinge лоссы. В качестве оптимизатора я тогда использовал SGD, с разными лернинг рейтами, но всегда с моментумом.
Ну и это в основном давало только 1.
Позже я догадался докрутить сюда weight_decay(что если я все правильно понял - просто регуляризация), что в принципе заставило мою модель учится. В целом, обычно я ставил этот параметр 1 или что то меньшее, все значения больше 1 обычно приводили к ухудшению. Я бы сказал, что это заставляло модель пытаться найти реальный закономерности, а не просто падать в лосс 0.5 с "киданием монетки"
Также немного улучшило мою модель, переход к softmax, и crossentopy лоссу. Но это скорее увеличило стабильность моей модели в зависимости от слоев, но в целом улучшения по пиковым скорам не дало.
Естественно, я часто добавлял нелинейность между линейными слоями - обычно это было ReLU, или LeakyRelu, чуть реже Tanh(по моим наблюдениям он работал сильно хуже).
В какой-то момент я добавил предпоследний слой в виде Linear(4, 2) ( в конце два, т.к. softmax выдает шансы принадлежности к классу на каждый класс), и это на самом деле дало какой-то минимальный буст.
Из оптимизаторов я кроме SGD пробовал Adam и Adagrad. Но, если я правильно понимаю как это все работает, я бы сказал, что "Хороший оптимизатор может сделать хорошую модель лучше, но не может сделать плохую модель хорошей"(Что вообще конечно вероятно не правда. Как минимум у нас на лекции был пример с седлом, где можно просто не упасть вниз, но всё же)
Dropout я тоже прикручивал. Это кстати модель в целом улучшало(по крайней мере если не ставить dropout после слоя с 4 входами. Там да, там это обычно делало все хуже).
В целом даже приблизится к бейслайну я так ни разу и не смог.
У меня была идея рекуррентной сети, с интуитивной логикой: передавать каждый раз нового героя, но как-то руки не дошли ее сделать.
Так что просто напишу две какие-нибудь сетки сюда, и перейду к следующему эксперименту

In [6]:
#Модель с сигмоидой в качестве функции активации, и лоссом в виде BCELoss
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
#train = pd.read_csv('json_train.csv')
#train = train.drop("Unnamed: 0", axis = 1)
X_train,Y_train, X_test, Y_test = preprocess_train(train, True)
train_dataset = MyTensorDataset(torch.FloatTensor(X_train.values), torch.FloatTensor(Y_train.values))
test_dataset = MyTensorDataset(torch.FloatTensor(X_test.values), torch.FloatTensor(Y_test.values))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=False,)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False,)
size = len(X_train.columns)

net = nn.Sequential(nn.Linear(size, size), nn.Linear(size, 1), nn.Sigmoid())

n_epoch = 20
criterion = nn.BCELoss()
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum= 0.1, weight_decay=1)
loss_history = []

for i in range(n_epoch):
    for x_batch, y_batch in train_dataloader:
        #print(x_batch)
        #y_batch = torch.transpose(y_batch, 0, 1)[0]
        #y_batch = y_batch[0]
        #print(y_batch)
        # Forward
        predictions = net(x_batch)
        #print(predictions)
        loss = criterion(predictions, y_batch)

        # Backward
        loss.backward()

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

        loss_history.append(loss.item())
    print(f'{i+1}/{n_epoch} current loss: {loss}')
    test_preds = np.array([])

for x_batch, y_batch in test_dataloader:
    predictions = net(x_batch)
    predictions = torch.transpose(predictions, 0, 1)[0]
    predictions = predictions.detach().numpy()
    test_preds = np.append(test_preds, predictions)

1/20 current loss: 0.0
2/20 current loss: 100.0
3/20 current loss: 0.0
4/20 current loss: 100.0
5/20 current loss: 100.0
6/20 current loss: 0.0
7/20 current loss: 0.0
8/20 current loss: 0.0
9/20 current loss: 0.0
10/20 current loss: 100.0
11/20 current loss: 100.0
12/20 current loss: 100.0
13/20 current loss: 100.0
14/20 current loss: 0.0
15/20 current loss: 100.0
16/20 current loss: 100.0
17/20 current loss: 100.0
18/20 current loss: 100.0
19/20 current loss: 100.0
20/20 current loss: 100.0


In [7]:
test_preds 

array([1.        , 1.        , 1.        , ..., 0.99999988, 1.        ,
       1.        ])

In [8]:
from sklearn.metrics import roc_auc_score
#print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, test_preds)}")

ROC-AUC score : 0.49515764667061685


In [13]:
#Softmax и Crossentropy модель
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = preprocess_train(train, True, True, True)
train_dataset = MyTensorDataset(torch.FloatTensor(X_train.values), torch.LongTensor(Y_train.values))
test_dataset = MyTensorDataset(torch.FloatTensor(X_test.values), torch.LongTensor(Y_test.values))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=False,)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True,)
size = len(X_train.columns)
net = nn.Sequential(nn.Linear(size, 200), nn.Linear(200, 50), nn.Linear(50, 4), nn.Linear(4, 2),
                    nn.Softmax())
n_epoch = 20
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=0.1)
loss_history = []

for i in range(n_epoch):
    for x_batch, y_batch in train_dataloader:
        #print(x_batch)
        y_batch = torch.transpose(y_batch, 0, 1)[0]
        #y_batch = y_batch[0]
        #print(y_batch)
        # Forward
        predictions = net(x_batch)
        #print(predictions)
        loss = criterion(predictions, y_batch)

        # Backward
        loss.backward()

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

        loss_history.append(loss.item())
    print(f'{i+1}/{n_epoch} current loss: {loss}')

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


1/20 current loss: 1.31326162815094
2/20 current loss: 0.41076546907424927
3/20 current loss: 0.31876614689826965
4/20 current loss: 0.45332223176956177
5/20 current loss: 0.7750725150108337
6/20 current loss: 0.4731188714504242
7/20 current loss: 1.31326162815094
8/20 current loss: 0.9056800603866577
9/20 current loss: 0.6332128047943115
10/20 current loss: 0.31364381313323975
11/20 current loss: 1.0419590473175049
12/20 current loss: 0.7242553234100342
13/20 current loss: 0.7931531667709351
14/20 current loss: 0.8484561443328857
15/20 current loss: 0.4020085036754608
16/20 current loss: 0.46867796778678894
17/20 current loss: 1.31326162815094
18/20 current loss: 0.4089151918888092
19/20 current loss: 0.3837886452674866
20/20 current loss: 0.9812914133071899


In [14]:
test_preds = test_by_nn(net, test_dataloader, False)

tensor([[4.2112e-05, 9.9996e-01],
        [0.0000e+00, 1.0000e+00],
        [2.4012e-04, 9.9976e-01],
        [4.5709e-01, 5.4291e-01],
        [3.6319e-01, 6.3681e-01],
        [9.6466e-04, 9.9904e-01],
        [7.3554e-02, 9.2645e-01],
        [7.8234e-02, 9.2177e-01]], grad_fn=<SoftmaxBackward0>)
tensor([[9.9967e-01, 3.3277e-04],
        [8.5477e-01, 1.4523e-01],
        [2.9000e-01, 7.1000e-01],
        [9.9766e-01, 2.3388e-03],
        [7.3842e-01, 2.6158e-01],
        [9.0401e-02, 9.0960e-01],
        [9.9967e-01, 3.3353e-04],
        [4.1847e-04, 9.9958e-01]], grad_fn=<SoftmaxBackward0>)
tensor([[3.4455e-01, 6.5545e-01],
        [3.4207e-01, 6.5793e-01],
        [2.3041e-02, 9.7696e-01],
        [3.5189e-02, 9.6481e-01],
        [3.2970e-02, 9.6703e-01],
        [9.9967e-01, 3.3200e-04],
        [4.0277e-01, 5.9723e-01],
        [0.0000e+00, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)
tensor([[5.1184e-01, 4.8816e-01],
        [9.2962e-01, 7.0377e-02],
        [2.8494e-04, 9.9972e-

In [15]:
test_preds 

array([4.21118202e-05, 0.00000000e+00, 2.40120993e-04, ...,
       5.78707010e-02, 7.53219524e-07, 5.08234382e-01])

In [16]:
from sklearn.metrics import roc_auc_score
print(f"ROC-AUC score : {roc_auc_score(Y_test, test_preds)}")

ROC-AUC score : 0.5061221988753858


#### Эксперимент 3 Линейная и Логистическая регрессия

Да. Иногда вернуться к простому видимо бывает полезно. На самом деле я написал это все немножко в другой хронологической последовательности: начал я с svm, потом перешел к нейросетям, недолго пописал их и перешел к логистической регрессии, но получил тогда чууууть чуть меньше бейслайна. Дело было в том, что я тогда не знал про predict_proba, из-за чего просто вернулся к нейросетям. После 3х-4х дней только нейросетей, я вернулся к идеи регрессии - и так как тогда я не знал про то что я могу и логистической регрессией предсказывать не только 0\1 но и шансы(я об этом догадывался, но не знал как, и почему-то просто не пошел искать. Хотя буквально пару днями позже просто загуглил и нашел),Я подумал почему бы не дать шанс обычной, линейной регрессии. И это, что на самом деле иронично, была первая модель что пробила у меня бейслайн. Да еще как - 0.795 на кегле(она еще тогда предсказывала значения меньше 0 и больше 1, но чисто ради формальности я залил еще одно решение где просто ручками после работы модели обрезал все значения не подходящие под формальное определение вероятности, но скор это, естественно, не поменяло). Иии вот после этого, я узнал о предикт проба. Так я и получил свой финальный скор - используя логистическую регрессию с препроцессингом в форме удаления аномально длинных игр, и фичей что я считал не значимыми.
p.s. забыл упомянуть - финальная линейная регрессия была с l2 регуляризацией

Давайте рассмотрим две мои модельки - линейную и финальную логистическую регрессию:

In [17]:
def preprocess_train(_data, is_remove_outliers, is_split = True, make_opposite_target = False):
    data = _data.copy()
    data = data.drop("match_id", axis = 1)
    if is_remove_outliers:
        data = delete_outliers(data)
    data = inner_preprocess(data)
    if is_split:
        return split_train(data, make_opposite_target)
    else:
        return data

In [18]:
# Линейная регрессия
from sklearn.linear_model import Ridge
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = preprocess_train(train, True)
model = Ridge()
model.fit(X_train, Y_train)

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.15175e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge()

In [19]:
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict(X_test))}")

Score : 0.25862184264063726
ROC-AUC score : 0.8053425123681386


In [21]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = preprocess_train(train, True)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, Y_train)

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

#### Достойные упоминания

Здесь я бы также хотел упомянуть деревья и бустинги. Я успел поработать с деревом решений, случайным лесом, а также градиентным бустингом и catboost. Но делал я это уже в последний день, и скорее из интереса - т.к. много был наслышан(слухи о catboost летели то там, то сям). Я особо с ними не экспериментировал, просто попробовал те данные, се данные, так просто посмотрел как они себя поведут при самых базовых параметрах. Но т.к. результатов я от них не добился, т.к. не выделил им достаточно времени, решил что вставлять код не буду. Вот так вот

#### Заключение

На самом деле, так получилось что я почти все что хотел бы видеть тут расписал в введении. Я рад тому как смог "поиграться" с разными моделями(буду честен, прямо как ребенок. На самом деле, тот факт что конкретно в учебной плоскости возможности кроме как на кеггле просто потрогать модели у нас нет(в дз мы все таки конкретно этим не занимаемся, а на семинарах мы все таки смотрим, интриги там не так много если пробовать самому после повторять и\или что-то менять)). За первое место - и за места в принципе я себе в этом кеггле бороться цели даже и не ставил. Единственное что меня прямо расстраивает - это то что я вообще ничего не добился нейросетями, и слабо понимаю почему. Сейчас смотря на это позади расписав всё всё для лабы я вероятно сказал бы что это проблема того, что я мало препроцессил данные - может быть если бы я подбирал фичи и по фичинжинерил - всё было бы лучше даже с теми моделями что я писал, но тогда мне казалось что как данные не препроцесь если сетка ничего не может - то она ничего не может. Во всяком случае, в этот раз я вполне наигрался, и теперь готов: в третьем кеггле я собираюсь идти за первым местом. Получиться ли у меня? Вряд ли. Но я явно постараюсь.

P.s. не думаю что это важно, но я пытался заносить важные вещи на одтельный репозиторий в гите. И хотя на самом деле я бы не хотел его оставлять, т.к. в силу того что делал его для себя названия и описания коммитов там не всегда правильные, да и ветки называются очень странно, да и в какой-то момент всё у меня там потекло и я даже днями мог не коммитить эксперименты с сетками, я всё же оставлю здесь ссылочку. В конце концов - вы же можете его просто не смотреть, раз уж я так плохо о нем отозвался. А хуже от его наличия вряд ли станет:

https://github.com/firefiruses/dotakaggle